# Replica project

### The data

In [1]:
# loading the metadata
%load_ext autoreload
%autoreload 2

import pandas as pd
from IPython.display import Image as Image2
import numpy as np

from torch import nn
import torchvision.models as models
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt

from urllib.request import urlretrieve 
import pickle
from glob import glob
from scipy import sparse

from torch.utils.data import DataLoader

from utils import *
from dataloader_replica import ReplicaDataset
from train_replica import train_replica
from model_replica import ReplicaNet

import umap

from utils import * 
path = '/home/guhennec/scratch/2021_Cini/TopologicalAnalysis_Cini/data/'
data_dir = '/scratch/students/schaerf/'



2022-03-17 10:50:46.447849: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')
metadata.head(5)

/tmp/ipykernel_1570752/2286784503.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(path + 'Cini_20210811.csv', sep=';')


,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,AuthorURL,AuthorComplemented,AuthorComplement,...,AuthorDeath,AuthorBirthLong,AuthorBirthLat,AuthorDeathLong,AuthorDeathLat,AuthorBirthCity,AuthorDeathCity,uid,CardboardURL,ImageURL
0,1A,1,ALLEGRINI Francesco,tca con penna acquarellata 148 x 203 mm,"ALLEGRINI, Francesco","Allegrini, Francesco",ulan:500115272,http://vocab.getty.edu/page/ulan/500115272,0,No complement,...,1690.0,NaN,NaN,NaN,NaN,NaN,NaN,253993c139284a45be233a13121ddeeb,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_1....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
1,1A,2,BAROCCI Federico,Studio di figura penna acquarellata (carboncin...,"BAROCCI, Federico","Barocci, Federico",ulan:500115210,http://vocab.getty.edu/page/ulan/500115210,0,No complement,...,1612.0,12.633333,43.716667,12.633333,43.716667,Urbino,Urbino,1323356994c24635a11fdcd9d5f9284a,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_2....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
2,1A,3,BASSANO Leandro,Scene penna su carta blu 171 x 219 mm,"BASSANO, Leandro","Bassano, Leandro",ulan:500015945,http://vocab.getty.edu/page/ulan/500015945,0,No complement,...,1622.0,11.733333,45.766667,12.326667,45.438611,Bassano del Grappa,Venice,a4268385f6384e61a3dd092bc6b8c083,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_3....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
3,1A,4,CAMPIGLI Massimo,Ragazza su una scala gessetto su carta bruna g...,"CAMPIGLI, Massimo","Campigli, Massimo",ulan:500029770,http://vocab.getty.edu/page/ulan/500029770,0,No complement,...,1971.0,13.400000,52.516667,6.639811,43.269316,Berlin,Var,550f368cdb4442aab4d5e2ada702d6ad,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_4....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...
4,1A,5,CARRACCI A attr,Madonna e Santi penna 291 x 205 mm,"CARRACCI, A. attr.",NaN,NaN,NaN,1,attr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6047de5547b643cb87491be925748bee,https://dhlabsrv4.epfl.ch/iiif_cini/1A%2F1A_5....,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...


In [3]:
metadata['uid'].nunique(), metadata.shape

(329480, (330002, 37))

In [4]:
# dropping duplicates

metadata = metadata.groupby('uid').first().reset_index()

In [5]:
# in jeanne's code
with open(path + 'save_link_data_2018_08_02.pkl', 'rb') as f:
    morpho_graph_complete = pickle.load(f)

# contains uid	img1	img2	type	annotated
morpho_graph_complete.head()

,uid,img1,img2,type,annotated
0,0f84770be893475581bebd8c0ce95704,4b6e5a18012e4974a43f6f652c9cad78,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:50.997512+00:00
1,5ade49fc2dc649b9947496a41e2decdb,384b9f017d5f43ba9dcfd92f2af830c4,e3329a91a266412fb1a0975d70894178,POSITIVE,2017-07-19 15:02:51.238615+00:00
2,bf4d99eaf7da4bd486f9007c90c177a5,94fe74bcebf9442fb5cd9a0327590a84,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:23.705228+00:00
3,b29ec3c83c5641f9b6afadb2409b6746,922fe92d61e744b1a9462c848a916bcf,726f47fb5bd94c5db8069334cc58020e,POSITIVE,2017-07-20 12:18:24.021703+00:00
4,ab27b55997b94bef9cca122829e9f64d,94fe74bcebf9442fb5cd9a0327590a84,922fe92d61e744b1a9462c848a916bcf,POSITIVE,2017-07-20 12:18:23.873097+00:00


In [6]:
morpho_graph_complete['type'].value_counts() # does positive mean similar?

DUPLICATE        114283
POSITIVE           6293
NON-DUPLICATE        60
PROPOSAL             27
Name: type, dtype: int64

In [7]:
morpho_graph_complete['uid'].nunique(), morpho_graph_complete.shape

(120663, (120663, 5))

In [8]:
positives = get_train_test_split(metadata, morpho_graph_complete)

In [9]:
positives = positives.groupby('uid_connection').first().reset_index()
print(positives.shape)
positives.head()

(2862, 45)


,uid_connection,index,uid,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,AuthorULAN,...,AuthorBirthCity,AuthorDeathCity,CardboardURL,ImageURL,img1,img2,type,annotated,cluster,set
0,002ca9c06a704c779e69d93cb4fadaa3,649,45588b4e33444ed8b34c523bac808bc4,158B,556,TIZIANO Vecellio (copia da),S.Mariá Maddalena. cm. 123 x 91,TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158B%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,45588b4e33444ed8b34c523bac808bc4,da54c106180449d1a52993e9a5bc17fd,POSITIVE,2018-07-24 14:58:44.559049+00:00,50,train
1,0038ab462149452b98a4d7359a06f9f4,1571,ae828b07e4284d80b652206ffb77278a,158C,120,TIZIANO Vecellio (copia da),"Morte di S. Pietro Martire. mm. 123,8 x 84,2",TIZIANO (copia da -),Titian,ulan:500031075,...,Belluno,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/158C%2F158...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,ae828b07e4284d80b652206ffb77278a,ea64207684694d379579998f88b7c29a,POSITIVE,2018-07-16 09:47:57.987515+00:00,51,test
2,00474897de044f7fb1ef16888dc198f3,1107,76b2c4aeab404bc79768117e3ddc62b1,47A,858,BELLINI Giovanni (cerchia di),Pietà.,"BELLINI, Giovanni (cerchia di -)","Bellini, Giovanni",ulan:500019244,...,Venice,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/47A%2F47A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,76b2c4aeab404bc79768117e3ddc62b1,b8dd2c0c2c7146e68e5bbb0aaf6cf803,POSITIVE,2018-03-14 10:46:45.603341+00:00,498,test
3,009c03c265b8418495da038bc3826bf7,1764,c5839e5220e5419e92d878396b0c04bb,135C,57,SEC XVI ex,Un soffitto. Part.: Leda e il cigno.,SEC. XVI ex.,None,None,...,None,None,https://dhlabsrv4.epfl.ch/iiif_cini/135C%2F135...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,450ee2aceb014709b23a93e1163937c5,c5839e5220e5419e92d878396b0c04bb,POSITIVE,2018-05-07 08:28:11.797723+00:00,92,train
4,00aeddb93a8a4120aa984eda6c822747,747,4d02860f58f2443ba4d1ef088756f854,21C,228,BASSANO Leandro,"La preparazione del pranzo. cm. 88,3 x 102,9","BASSANO, Leandro.","Bassano, Leandro",ulan:500015945,...,Bassano del Grappa,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/21C%2F21C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,4d02860f58f2443ba4d1ef088756f854,27bad0f213a342c3987e8cec61e40550,POSITIVE,2018-03-23 15:41:53.094226+00:00,218,train


## Getting a subset of the data

In order to select a subset of the data, we would like to vary spatially and temporally, for it to be suitable to the data in the morphograph and to be comprehensive of what we want to study.


In [10]:
subset_max = metadata[metadata['BeginDate'] > 1550][metadata['BeginDate'] < 1650][metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()
subset_max.shape

/tmp/ipykernel_1570752/3952239037.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  subset_max = metadata[metadata['BeginDate'] > 1550][metadata['BeginDate'] < 1650][metadata['AuthorDeathCity'].isin(['Parma', 'Bologna', 'Ravenna', 'Verona', 'Ferrara'])].reset_index()


(3302, 38)

In [49]:
list_downloaded = [file.split('/')[-1].split('.')[0] for file in glob(data_dir + 'subset/*')]
list_downloaded[0]

'383d8fb371ae4aa9bd3819fb973f0ac9'

In [50]:
len(list_downloaded)

4593

In [13]:
del morpho_graph_complete
del metadata

In [14]:
uniques = subset_max[subset_max['uid'].isin(list_downloaded)].groupby('uid').first().reset_index()


print(uniques.shape)

(3301, 38)


In [26]:
data = pd.concat([positives, uniques], axis=0).reset_index()
print(data.shape)

data.to_csv(data_dir + 'full_data.csv')

(6163, 46)


In [27]:
data.sample(5)

,level_0,uid_connection,index,uid,Drawer,ImageNumber,Author,Description,AuthorOriginal,AuthorULANLabel,...,AuthorBirthCity,AuthorDeathCity,CardboardURL,ImageURL,img1,img2,type,annotated,cluster,set
4624,1762,NaN,174881,87f9919c81fb4c1da4365e38f0dc658e,9A,187,GUERCINO (BARBIERI Giovanni Francesco),Paesaggio con fiume 281 x 409,GUERCINO (i1-),Guercino,...,Cento,Bologna,https://dhlabsrv4.epfl.ch/iiif_cini/9A%2F9A_18...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN,NaN,NaN,NaT,NaN,NaN
3390,528,NaN,54269,2a12ab793be84eff98761c2c173682f0,22B,115,CARRACCI Lodovico,"Allegoria della Storia. Penna, acquerello grig...","CARRACCI, Lodavico.","Carracci, Lodovico",...,Bologna,Bologna,https://dhlabsrv4.epfl.ch/iiif_cini/22B%2F22B_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN,NaN,NaN,NaT,NaN,NaN
3771,909,NaN,90767,461cf0273a644cf29172476a6f63da3c,28B,70,GUERCINO (il),Busto femminile di profilo.,GUERCINO.II .,Guercino,...,Cento,Bologna,https://dhlabsrv4.epfl.ch/iiif_cini/28B%2F28B_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,NaN,NaN,NaN,NaT,NaN,NaN
2088,2088,be85ba5052a04647aba145685bb27399,135,0df535371e6a4e6c8152b858d00adc36,43A,208,PIETRO MONACO (inc ) STROZ2I Bernardo (inv ),David con la testa di Golia,"PIETRO MONACO (inc.) STROZ2I, Bernardo (inv.)","Guardi, Francesco",...,Venice,Venice,https://dhlabsrv4.epfl.ch/iiif_cini/43A%2F43A_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,0df535371e6a4e6c8152b858d00adc36,1e1398112e564c5bb0223c0f64554f91,POSITIVE,2018-03-16 18:39:01.512580+00:00,661.0,train
234,234,13f9b8a429b14321b20a33ae3019bd20,130,0eef8204802d4d0ea8476f99d60ad6a3,21C,131,SASSOFERRATO (il),"La Madonna col Bambino. cm. 70 x 58,5",SASSOFERRATO (IL),Sassoferrato,...,Ancona,Rome,https://dhlabsrv4.epfl.ch/iiif_cini/21C%2F21C_...,https://dhlabsrv4.epfl.ch/iiif_replica/cini%2F...,a7da74e233084c39a68f53ad787d407b,0eef8204802d4d0ea8476f99d60ad6a3,POSITIVE,2018-03-02 18:44:38.643686+00:00,590.0,train


### Creating embeddings of all the images

In [28]:
model = models.resnet50(pretrained=True)
#print(model)

In [29]:
newmodel = torch.nn.Sequential(
    *(list(model.children())[:-2]), nn.AdaptiveMaxPool2d((1,1))
)
#print(newmodel)


In [30]:
img = preprocess_image(data_dir + 'subset/' + data['uid'][0] + '.jpg')
embedding = get_embedding(img, newmodel).squeeze(3).squeeze(1).squeeze(0)

In [31]:
embeddings = [get_embedding(preprocess_image(data_dir + 'subset/' + uid + '.jpg'), newmodel).squeeze(3).squeeze(1).squeeze(0) for uid in tqdm(data['uid'].unique())]

100%|██████████| 4357/4357 [16:40<00:00,  4.35it/s]


In [32]:
embeddings_new = get_lower_dimension(embeddings)

In [33]:
embeddings[0].shape, embeddings_new[0].shape 

((2048,), (100,))

### Finding most similar embeddings

In [34]:
tree = make_tree_list(embeddings_new)

In [60]:
train_test = data[data['set'].notnull()].drop(columns=['level_0']).reset_index()

Cs = []
for i in tqdm(range(train_test.shape[0])):
    list_theo = list(train_test[train_test['img1'] == train_test['uid'][i]]['img2']) + list(train_test[train_test['img2'] == train_test['uid'][i]]['img1']) + [train_test['uid'][i]]
    list_ = find_most_similar_list(train_test['uid'][i], tree, embeddings_new, list(data['uid'].unique()), list_theo)
    Cs.append(list_)

len(Cs)


100%|██████████| 2862/2862 [00:10<00:00, 286.12it/s]


2862

In [58]:
print(train_test[train_test['img1'].isin(list_downloaded)].shape)
print(train_test[train_test['img2'].isin(list_downloaded)].shape)

(2299, 47)
(2256, 47)


In [61]:
train_test['C'] = Cs

train_test = train_test[train_test['img1'].isin(list_downloaded)]
train_test = train_test[train_test['img2'].isin(list_downloaded)]

final = train_test[['img1', 'img2', 'C', 'set']].explode('C')
final.columns = ['A', 'B', 'C', 'set']
print(final.shape)
final.sample(5)

(31709, 4)


,A,B,C,set
1771,6fcf1af179fa47488d40564e80856a1e,e100fa4b1d1e4abab65256ac310ab543,6bf773f2e04f4c919ccd6f0b76df2b15,train
1539,ef25d7d6549547b59945a61ae6849d3d,272cd39493464a878264554be785e9db,92fc36e4d04743ec97e441342acd29f1,test
63,59fb95590dda4ff58e63dc0df43e70c3,a043a8ccb7994ef68179ad66929143af,756a2003514740448ffb37eec9351177,train
289,383cd750b1cc425089ff3cf3d2c81df1,8b47beb1f1a6438993f3988e880cfa9c,45d519d9ba884c948b2a0abf1a692599,test
581,3d7b289477ca4765a6932bd85cf51fec,246ca4d9a3324ad2aad8ec6a4e1c6769,4ced1a6287c1417db8b41e79e1a04c79,test


In [62]:
final[final['C'].isin(list_downloaded)].shape

(31709, 4)

In [63]:
final[final['set'] == 'train'].reset_index().to_csv(data_dir + 'abc_train.csv')
final[final['set'] == 'test'].reset_index().to_csv(data_dir + 'abc_test.csv')
